In [ ]:
'''
관련 Github, 링크
github: https://github.com/IntelRealSense/librealsense/blob/master/wrappers/python/examples/align-depth2color.py
공식 Docs: https://dev.intelrealsense.com/docs/tensorflow-with-intel-realsense-cameras
'''
# doc의 공식 코드


# depth color map이랑 rgb카메라 스트리밍 공식 코드
## License: Apache 2.0. See LICENSE file in root directory.
## Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

###############################################
##      Open CV and Numpy integration        ##
###############################################

import pyrealsense2 as rs
import numpy as np
import cv2

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.imshow('Depth Image', depth_image) # 라이언 추가
        if cv2.waitKey(1) & 0xff == ord('q'):
            cv2.destroyAllWindows()
            break

finally:
    # Stop streaming
    pipeline.stop()

In [65]:
# 240514 라이언 제작 모듈
import pyrealsense2 as rs
import numpy as np
import cv2

class real_sense:
    def __init__(self):
        '''
        Real Sense 카메라 연결하여 파이썬으로 RGB 이미지와 Depth Map을 불러오는 Class.
        공식 문서를 참고하여 사용하기 쉽게 Class의 형태로 제작함
        '''
        # Configure depth and color streams
        self.pipeline = rs.pipeline()
        config = rs.config()

        # Get device product line for setting a supporting resolution
        pipeline_wrapper = rs.pipeline_wrapper(self.pipeline)
        pipeline_profile = config.resolve(pipeline_wrapper)
        device = pipeline_profile.get_device()
        device_product_line = str(device.get_info(rs.camera_info.product_line))

        found_rgb = False
        for s in device.sensors:
            if s.get_info(rs.camera_info.name) == 'RGB Camera':
                found_rgb = True
                break
        if not found_rgb:
            print("The demo requires Depth camera with Color sensor")
            exit(0)

        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

        # Start streaming
        self.pipeline.start(config)
        print('Real Sense 카메라 수신 설정 완료')

    def get_cam(self):
        '''
        RealSense 카메라에서 frame들을 가져옴.
        이 frame들에서 RGB이미지, Depth맵 등 다양한 이미지가 추출됨
        frame이 return되지는 않고, 내부 변수로 활용됨
        '''
        self.frames = self.pipeline.wait_for_frames()
    
    def get_color_img(self):
        '''
        get_cam 실행 이후에 실행되어야 에러가 발생하지 않음
        self.frames라는 변수가 존재해야 실행 가능한 로직이기 때문임
        '''
        color_frame = self.frames.get_color_frame()
        if color_frame:
            self.color_image = np.asanyarray(color_frame.get_data())
            return self.color_image
        else:
            print('get_color_img 에러 발생!')
        
    def get_depth_img(self):
        '''
        get_cam 실행 이후에 실행되어야 에러가 발생하지 않음
        self.frames라는 변수가 존재해야 실행 가능한 로직이기 때문임
        '''
        depth_frame = self.frames.get_depth_frame()
        if depth_frame:
            self.depth_image = np.asanyarray(depth_frame.get_data())
            return self.depth_image
        else:
            print('get_depth_img 에러 발생!')

    def get_depth_color_map(self):
        '''
        self.depth_image로부터 color_map을 만들어서 반환함
        반드시 get_depth_img 함수가 실행되어야 에러가 발생하지 않음
        self.depth_image가 존재해야 실행 가능한 로직이기 때문임
        '''
        self.depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
        return self.depth_colormap
    
    def concat_all(self):
        '''
        위에서 만든 color_img, depth_map, depth_color_map 3개를 가로 Concat하여 반환
        위 3개 이미지가 모두 존재해야 에러가 발생하지 않음
        위 3개 함수를 선언하지 않고 코드를 실행하면 이전 frame의 이미지가 concat될 것임(주의 필요)
        '''
        # 깊이 이미지의 최대값을 확인하고, 8비트 형식으로 스케일 조정
        scaled_depth_image = cv2.convertScaleAbs(self.depth_image, alpha=(255.0/np.max(self.depth_image)))
        depth_3d = cv2.cvtColor(scaled_depth_image, cv2.COLOR_GRAY2BGR) # 1차원인 depth 이미지를 3차원으로 변경함
        concatenated_image = cv2.hconcat([self.color_image, depth_3d, self.depth_colormap])
        return concatenated_image

In [66]:
RealSense = real_sense()
while True:
    RealSense.get_cam() # 카메라 수신
    color_img = RealSense.get_color_img()
    depth_img = RealSense.get_depth_img()
    depth_color_map = RealSense.get_depth_color_map()
    cv2.imshow('Real Sense Streaming[color_img, depth_img, depth_color_map]', RealSense.concat_all())
    cv2.imshow('depth map', RealSense.depth_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

Real Sense 카메라 수신 설정 완료


In [75]:
from ultralytics import YOLO
import numpy as np

# YOLOv8 연동
class YOLOv8:
    def __init__(self):
        '''
        YOLOv8모델 로드 및 인퍼런스 준비
        '''
        self.model = YOLO('./yolov8n.pt')
        print('YOLOv8 모델 로드 완료')
        self.cls_list = 'none'
    
    def run(self, img, depth_map):
        '''
        img: YOLOv8로 추론할 이미지 입력
        추론 결과인 dic_list 반환(bbox[x1, y1, x2, y2], conf, class_name 포함)
        depth_map도 같이 받아서 물체의 거리 반환
        '''
        self.img = img
        results = self.model.predict(source = cv2.cvtColor(img, cv2.COLOR_BGR2RGB), verbose = False)[0]
        if self.cls_list == 'none':
            cls_list = results.names
        results = results.boxes.data.tolist()
        self.dic_list = []
        for result in results:
            x1, y1, x2, y2, conf, cls = int(result[0]), int(result[1]), int(result[2]), int(result[3]), float(result[4]), int(result[5])
            # depth 추출
            small_x1, small_y1, small_x2, small_y2 = self.make_small_bbox(x1, y1, x2, y2)
            depth = self.calculate_average_depth(depth_map, small_x1, small_y1, small_x2, small_y2)
            self.dic_list.append({'bbox':[x1, y1, x2, y2], 'conf':conf, 'class_name':cls_list[cls], 'depth':depth})
        return self.dic_list
    
    def draw(self):
        '''
        추론한 결과를 이미지에 그려서 반환해주는 함수.
        내부 변수로 img와 dic_list를 가지고 있기 때문에 인자로 넣어줄 필요 없음.
        그려진 이미지 반환. 이 함수를 실행하는 순간 내부 self.img는 그려진 이미지로 바뀜
        '''
        for dic in self.dic_list:
            cv2.rectangle(self.img, (dic['bbox'][0], dic['bbox'][1]), (dic['bbox'][2], dic['bbox'][3]), (0,0,255), 2)
            text = f'{dic["class_name"]}:{round(dic["conf"], 2)}'
            cv2.putText(self.img, text, (dic['bbox'][0]-5, dic['bbox'][1]-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        return self.img

    def make_small_bbox(x1, y1, x2, y2):
        '''
        bbox를 넣으면 일정 비율로 축소해주는 함수
        '''
        ratio = 0.1
        center_x, center_y = (x1+x2)/2, (y1+y2)/2
        x, y = x2-x1, y2-y1
        small_x, small_y = x*ratio, y*ratio
        new_x1 = center_x - (small_x/2)
        new_y1 = center_y - (small_y/2)
        new_x2 = new_x1 + small_x
        new_y2 = new_y1 + small_y
        return int(new_x1), int(new_y1), int(new_x2), int(new_y2)
        

    def calculate_average_depth(depth_map, x1, y1, x2, y2):
        """
        지정된 bounding box 내에서 평균 깊이를 계산하는 함수.
        
        Parameters:
        depth_image (numpy array): 깊이 이미지 (2D numpy array 형태).
        x1, y1 (int): bounding box의 왼쪽 상단 좌표.
        x2, y2 (int): bounding box의 오른쪽 하단 좌표.
        
        Returns:
        float: 계산된 평균 깊이 값. 0을 제외한 깊이 값의 평균.
        """
        # bounding box 영역에서 깊이 데이터를 추출
        depth_region = depth_image[y1:y2, x1:x2]
        
        # 0을 제외한 깊이 값만을 사용하여 평균 계산
        if np.count_nonzero(depth_region) > 0:
            average_depth = np.mean(depth_region[depth_region > 0])
            return average_depth
        else:
            print("해당 영역에 유효한 깊이 데이터가 없습니다.")
            return None



In [76]:
RealSense = real_sense()
Yolov8 = YOLOv8()
while True:
    RealSense.get_cam() # 카메라 수신
    color_img = RealSense.get_color_img()
    depth_map = RealSense.get_depth_img()
    Yolov8.run(color_img, depth_map)
    draw_img = Yolov8.draw()
    cv2.imshow('YOLOv8 Inference', draw_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

Real Sense 카메라 수신 설정 완료
YOLOv8 모델 로드 완료


TypeError: make_small_bbox() takes 4 positional arguments but 5 were given